# Project 6: An Assembler for Our Computer

In [2]:
import Project6IO as IO #Contains nextline(), which returns the next line from the keyboard.

line = "" #This string will store the next line from the file.

#Some variables (registers) to store different pieces of the next command.
nextsym=""
nextdest=""
nextcomp=""
nextjump=""
nexttype = ""

#Outputs True if there are more commands.
# Otherwise outputs false.
def hasMoreCommands():
    global line
    if line != "EOF":
        return True
    else:
        return False


#Populates the state variables based on the current line,
# then reads the next line from the file.
def advance():
    #This lists the state (temporal) variables we'll use. Don't change it!
    global line, nextsym, nexttype, nextjump, nextcomp, nextdest
    
    #store the next line in the buffer.
    line=IO.nextLine()
    if line == "EOF":
        return
    
    #This block is like a mux for determining the next symbol, if any.
    if line[0] == '@':
        nextsym = line[1:]  #like nextline [1..end], chops off the @
        nexttype = "A"
    elif line[0] == '(': 
        nextsym = line[1:line.find(')')]
        nexttype = "L"
    else:
        nextsym = line[0:line.find('=')]
        nexttype = "C"
        
    #This block is like a mux for determining the next destination, if any.
    #compidx will store the start of the computation when the block is finished.
    compidx = 0
    if line.find('=') != -1:
        nextdest = line[0:line.find('=')]
        compidx = IO.nextidx
    else:
        nextdest = ''
        
    #This block determines both the next computation, and the next jump.
    if line.find(';') == -1:
        nextcomp = line[line.find('=')+1:]
        nextjump = ''
    else:
        nextcomp = line[line.find('=')+1:line.find(';')]
        nextjump = line[line.find(';')+1:]
    
    return 

def commandType():
    global nexttype
    if nexttype == "A":
        return "A_COMMAND"
    elif nexttype == "C":
        return "C_COMMAND"
    else:
        return "L_COMMAND"

def symbol():
    global nextsym
    #This block will trigger a fault if this subroutine is called unexpectedly.
    if nextsym == "":
        raise ValueError("No symbol on this line!")
    return nextsym


def dest():
    global nextdest
    if nextdest != "":
        return nextdest
    else:
        return "null"
    
def comp():
    global nextcomp
    if nextcomp == "":
        raise ValueError("No computation on this line!")
    return nextcomp

def jump():
    global nextjump
    if nextjump == "":
        return "null"
    else:
        return nextjump
    
    

In [3]:
#The Code Module

def destCode(s):
    if s == "null":
        return "000"
    elif s == "M":
        return '001'
    elif s == "D":
        return '010'
    elif s == "MD":
        return '011'
    elif s == "A":
        return '100'
    elif s == "AM":
        return '101'
    elif s == "AD":
        return '110'
    elif s == "AMD":
        return '111'
    else:
        raise ValueError("Error: destination code not recognized.")
        return ""

def jumpCode(s):
    if s == "null":
        return "000"
    elif s == "JGT":
        return '001'
    elif s == "JEQ":
        return '010'
    elif s == "JGE":
        return '011'
    elif s == "JLT":
        return '100'
    elif s == "JNE":
        return '101'
    elif s == "JLE":
        return '110'
    elif s == "JMP":
        return '111'
    else:
        raise ValueError("Error: Jump code not recognized")
        
def compCode(s):
    if s == "0":
        return "0101010"
    elif s == "1":
        return "0111111"
    elif s == "-1":
        return "0111010"
    elif s == "D":
        return "0001100"
    elif s == "A":
        return "0110000"
    elif s == "M":
        return "1110000"
    elif s == "!D":
        return "0001101"
    elif s == "!A":
        return "0110001"
    elif s == "!M":
        return "1110001"
    elif s == "-D":
        return "0001111"
    elif s == "-A":
        return "0110011"
    elif s == "-M":
        return "1110011"
    elif s == "D+1":
        return "0011111"
    elif s == "A+1":
        return "0110111"
    elif s == "M+1":
        return "1110111"
    elif s == "D-1":
        return "0001110"
    elif s == "A-1":
        return "0110010"
    elif s == "M-1":
        return "1110010"
    elif s == "D+A":
        return "0000010"
    elif s == "D+M":
        return "1000010"
    elif s == "D-A":
        return "0010011"
    elif s == "D-M":
        return "1010011"
    elif s == "A-D":
        return "0000111"
    elif s == "M-D":
        return "1000111"
    elif s == "D&A":
        return "0000000"
    elif s == "D&M":
        return "1000000"
    elif s == "D|A":
        return "0010101"
    elif s == "D|M":
        return "1010101"
    else:
        raise ValueError("Error: Computation code " + s + " not recognized!")
        return ""
        
    

In [4]:
#Assembler for Symbol-less Programs
IO.setFile('../add/Add.asm')

advance()
while hasMoreCommands():
    if commandType() == "A_COMMAND":
        IO.printAddress(nextsym)
    elif commandType() == "C_COMMAND":
        IO.printCommand('111', compCode(comp()), destCode(dest()), jumpCode(jump()))
    else:
        raise ValueError("L Commands not yet implemented!")
    advance()

0000000000000010
1110110000010000
0000000000000011
1110000010010000
0000000000000000
1110001100001000


In [5]:
#SymbolTable Module

table = {} #an empty lookup table.

def init():
    table = {}

def addEntry(symbol, address):
    table[symbol] = address

def contains(symbol):
    if symbol in table:
        return table[symbol]
    else:
        return None

def getAddress(symbol):
        return table[symbol]
    

In [6]:
#Assembler for Programs with Symbols

def populateTable():
    init()
    addEntry("SP", 0)
    addEntry("LCL", 1)
    addEntry("ARG", 2)
    addEntry("THIS", 3)
    addEntry("THAT", 4)
    addEntry("R0", 0)
    addEntry("R1", 1)
    addEntry("R2", 2)
    addEntry("R3", 3)
    addEntry("R4", 4)
    addEntry("R5", 5)
    addEntry("R6", 6)
    addEntry("R7", 7)
    addEntry("R8", 8)
    addEntry("R9", 9)
    addEntry("R10", 10)
    addEntry("R11", 11)
    addEntry("R12", 12)
    addEntry("R13", 13)
    addEntry("R14", 14)
    addEntry("R15", 15)
    addEntry("SCREEN", 16384)
    addEntry("KBD", 24576)

#You can remove the '#' from the front of the lines below when you're ready to test larger files.
IO.setFile("../pong/Pong.asm") 
IO.setSaveFile()

populateTable()
linecounter = 0
advance()
while hasMoreCommands():
    #
    if commandType() == "A_COMMAND":
        linecounter += 1
    elif commandType() == "L_COMMAND":
        addEntry(line[1:-1],linecounter)
    else:
        linecounter += 1
    advance()
    
IO.rewind()
line = ""
memcounter = 16
advance()
while hasMoreCommands():
    #
    if commandType() == "A_COMMAND":
        if contains(symbol()) == None and symbol().isdecimal() == False:
            addEntry(symbol(), memcounter)
            memcounter = memcounter + 1
        if symbol().isdecimal():
            IO.printAddress(symbol())
        else:
            IO.printAddress(getAddress(symbol()))
    
    elif commandType() == "C_COMMAND":
        IO.printCommand('111', compCode(comp()), destCode(dest()), jumpCode(jump()))
    elif commandType() == "L_COMMAND":
        pass #Don't need to do anything here.
    else:
        raise ValueError("Unknown Command Type!")
    advance()
